In [47]:
from ultralytics import YOLO
import easyocr
import cv2
import pandas as pd
import time
from datetime import datetime

In [2]:
npr_model = YOLO('anpr_best.pt')

In [10]:
image = 'image_reading_test.jpeg'
results = npr_model.predict(image)


image 1/1 e:\6th semester\Artificial Neural Networks\PROJECT\image_reading_test.jpeg: 256x320 1 Number Plate, 45.6ms
Speed: 8.5ms preprocess, 45.6ms inference, 9.6ms postprocess per image at shape (1, 3, 256, 320)


In [11]:
# Extract bounding boxes, classes, names, and confidences
boxes = results[0].boxes.xyxy.tolist()
classes = results[0].boxes.cls.tolist()
names = results[0].names
confidences = results[0].boxes.conf.tolist()

print(boxes)

[[1518.7303466796875, 1802.0206298828125, 2038.082763671875, 2240.0078125]]


In [36]:

def extract_number_plate_text(image_path, model):
  """
  Extracts and returns the text from a number plate in an image using YOLOv8 (assumed) detections.

  Args:
      image_path (str): Path to the image file.

  Returns:
      str: The recognized text of the number plate, or None if an error occurs.
  """

  try:
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Check if image loading was successful
    if image is None:
      print(f"Error: Could not read image '{image_path}'.")
      return None

    # Assuming "results" contains YOLOv8 detections in the expected format
    # (Modify this section to integrate with your YOLOv8 model output)
    npr_model = YOLO('anpr_best.pt')
    results = model.predict(image)
    boxes = results[0].boxes.xyxy.tolist()  # Get bounding boxes
    reader = easyocr.Reader(['en'], gpu = True)
    for bounding_box in boxes:
      try:
        # Extract coordinates using list indexing
        xmin = int(bounding_box[0])
        ymin = int(bounding_box[1])
        xmax = int(bounding_box[2])
        ymax = int(bounding_box[3])
        cropped_image = image[ymin:ymax, xmin:xmax]
        gray_img = cv2.cvtColor(cropped_image, cv2.COLOR_RGB2GRAY)
        # Create easyocr Reader with appropriate language code

        # Perform text recognition on the cropped image
        results = reader.readtext(gray_img)

        for result in results:
          text = result[0]
          confidence = result[1]
          # Return the first recognized text with confidence above a threshold (optional)
          if float(confidence) > 0.1:  # Adjust threshold as needed
            return text

      except (IndexError, ValueError):
        print(f"Error processing bounding box: {bounding_box}")

  except (AttributeError, IndexError):
    print("Error: Could not access bounding boxes from YOLOv8 output.")

  return None  # Return None if no text is recognized or an error occurs


In [39]:
# Assuming you have your YOLOv8 model integrated
image_path = '01413.jpg'  # Replace with your image path
recognized_text = extract_number_plate_text(image_path, npr_model)

if recognized_text:
  print(f"Number plate text: {recognized_text}")
else:
  print("No number plate text recognized.")


0: 224x320 1 Number Plate, 107.4ms
Speed: 0.0ms preprocess, 107.4ms inference, 27.2ms postprocess per image at shape (1, 3, 224, 320)
Number plate text: [[1, 3], [29, 3], [29, 17], [1, 17]]


In [40]:
reader = easyocr.Reader(['en'], gpu=True)
img = cv2.imread("01413.jpg")
results = npr_model(img)
boxes = results[0].boxes.xyxy.tolist()
classes = results[0].boxes.cls.tolist()
names = results[0].names
confidences = results[0].boxes.conf.tolist()

def perform_ocr_on_image(img, coordinates):
    x, y, w, h = map(int, coordinates)
    cropped_img = img[y:h, x:w]

    gray_img = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2GRAY)
    results = reader.readtext(gray_img)

    text = ""
    for res in results:
        if len(results) == 1 or (len(res[1]) > 6 and res[2] > 0.2):
            text = res[1]

    return str(text)


text = perform_ocr_on_image(img, boxes[0])
print("NP:", text)



0: 224x320 1 Number Plate, 18.9ms
Speed: 0.0ms preprocess, 18.9ms inference, 8.4ms postprocess per image at shape (1, 3, 224, 320)
NP: 4453


In [58]:
# Create an empty DataFrame with columns 'Text' and 'Timestamp' if it doesn't exist
from datetime import datetime

def add_entry(entry_dict, license_plate):

  timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  entry_dict['License Plate'].append(license_plate)
  entry_dict['Timestamp'].append(timestamp)

# Example usage
entry_dict = {'License Plate': [], 'Timestamp': []}
add_entry(entry_dict, "ABC123")
add_entry(entry_dict, "DEF456")

print(entry_dict)



{'License Plate': ['ABC123', 'DEF456'], 'Timestamp': ['2024-05-07 16:38:47', '2024-05-07 16:38:47']}


In [60]:
# Create a DataFrame from the dictionary
df = pd.DataFrame(entry_dict)
df

,License Plate,Timestamp
0,ABC123,2024-05-07 16:38:47
1,DEF456,2024-05-07 16:38:47


In [ ]:
from datetime import datetime
import pandas as pd

def add_entry(entry_gate_df, license_plate):
  """Adds a new entry with the given license plate and current timestamp to the DataFrame.

  Args:
      entry_gate_df: The DataFrame for storing entry gate data.
      license_plate: The license plate for the new entry.
  """
  timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  new_entry = {'License Plate': license_plate, 'Timestamp': timestamp}
  entry_gate_df = entry_gate_df.append(new_entry, ignore_index=True)  # Add entry to DataFrame

  # Try to read existing data from Excel (avoid overwriting on first run)
  try:
    existing_data = pd.read_excel("entry_gate_data.xlsx")
    entry_gate_df = entry_gate_df.append(existing_data, ignore_index=True)  # Combine existing and new data
  except FileNotFoundError:
    pass  # Ignore error on first run, new file will be created

  # Save DataFrame to Excel
  entry_gate_df.to_excel("entry_gate_data.xlsx", index=False)


# Example usage
entry_gate_df = pd.DataFrame(columns=['License Plate', 'Timestamp'])  # Initialize empty DataFrame
add_entry(entry_gate_df, "ABC123")

# Simulate reading more license plates (replace with your actual reading logic)
# ...

